In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import vectorbt as vbt

In [3]:
# Download the data
data = yf.download('SPY', start='2022-01-01', auto_adjust=True)
data.columns = data.columns.droplevel(1)

# Calculate the SMA
data['SMA50'] = data['Close'].rolling(window=50).mean()

# Determine the position state (1 for long, 0 for flat)
data['Position'] = np.where(data['Close'] > data['SMA50'], 1, 0)

# Calculate the signal (1 for buy, -1 for sell, 0 for hold)
data['Signal'] = data['Position'].diff()

# Display the rows where a trade signal occurred
print("Trade Signals (1 = Buy, -1 = Sell):")
print(data[data['Signal'] != 0].head())

[*********************100%***********************]  1 of 1 completed

Trade Signals (1 = Buy, -1 = Sell):
Price            Close        High         Low        Open     Volume  \
Date                                                                    
2022-01-03  453.210388  453.343222  449.548364  451.872697   72668200   
2022-03-18  423.032745  423.356307  416.085625  416.827920  106345500   
2022-04-11  418.655090  423.489520  418.150711  422.642527   89770500   
2022-04-13  421.881165  422.642482  416.675574  416.856392   74070400   
2022-04-14  416.628052  423.232587  416.523354  422.109604   97869500   

Price            SMA50  Position  Signal  
Date                                      
2022-01-03         NaN         0     NaN  
2022-03-18  419.555724         1     1.0  
2022-04-11  419.390379         0    -1.0  
2022-04-13  419.038951         1     1.0  
2022-04-14  418.693622         0    -1.0  


## 2. Prepare Signals for Vectorbt
Vectorbt's `Portfolio.from_signals` function prefers separate boolean (True/False) inputs for entries and exits. We can easily create these from your `Signal` column.

In [4]:
# Create entry signals where Signal is 1 (buy)
entries = data['Signal'] == 1

# Create exit signals where Signal is -1 (sell)
exits = data['Signal'] == -1

## 3. Run the Backtest 🚀
Now, you can feed your price data and signals directly into Vectorbt's portfolio simulator. This one function handles all the complex calculations of executing trades, tracking equity, and calculating performance metrics.

In [5]:
# Run the Vectorbt backtest
portfolio = vbt.Portfolio.from_signals(
    close=data['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,  # Start with $100,000
    freq='D'           # Use daily frequency for calculations
)

# Print the performance statistics
print("\n--- Backtest Performance ---")
print(portfolio.stats())


--- Backtest Performance ---
Start                                2022-01-03 00:00:00
End                                  2025-10-13 00:00:00
Period                                 948 days 00:00:00
Start Value                                     100000.0
End Value                                   134470.38992
Total Return [%]                                34.47039
Benchmark Return [%]                           46.298495
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                               20.259715
Max Drawdown Duration                  380 days 00:00:00
Total Trades                                          28
Total Closed Trades                                   27
Total Open Trades                                      1
Open Trade PnL                              21853.794759
Win Rate [%]                                   25.925926
Best Trade [%]                                 16.880766
W

## 4. Visualize the Results 📊
One of the best features of Vectorbt is its powerful plotting. You can visualize the trades on a chart along with your SMA indicator to see exactly how the strategy performed.

In [6]:
# Create the plot
fig = portfolio.plot(subplots=['price', 'positions'])

# Add the SMA50 indicator to the price subplot
fig.add_scatter(
    x=data.index, 
    y=data['SMA50'], 
    name='SMA50', 
    row=1, col=1
)

fig.show()

KeyError: 'price'